## Graph & Sessions
### 1.简介
Tensorflow使用数据流图来表示你的计算模型，具体来说，就是每个单独操作之间的依赖关系。

如果你打算直接使用低层API来构造你的模型的话，本篇tutorial对你会有很大帮助。高层API例如tf.estimator.Estimator及Keras对终端用户隐藏了Graph和Session的细节，但如果你想了解这些API是如何实现的话，这篇tutorial对你也许也有很大帮助。

数据流是针对并行编程而言一种常用的编程模型，在一个数据流图中，节点表示计算单元，边表示数据的流量（被使用以及被创建），例如，在Tensorflow图中，tf.matmul操作(在图中表示为一个节点)将会使用两个矩阵(两个流入的边)以及产生一个输出的矩阵(流出的边)。

或者，一个实际的神经网络例子：
<img src="https://www.tensorflow.org/images/tensors_flowing.gif?hl=zh-cn">

### 2.tf.Graph
一个tf.Graph主要包含以下两类相关信息：
* Graph的结构：就是刚刚说的节点和边啦
* Graph集合：Tensorflow提供了存储Graph中元数据的通用机制。这个tf.add_to_collection方法使你可以通过键值来操作对象，以及tf.get_collection方法使你可以与某个键值相关的所有对象。大部分Tensorflow的library都使用以下特性：当你创建一个Variable时，该Variable将会自动被加入到名为“global variables”和“rainable variables”的集合中，当不久后你创建tf.train.Saver或者tf.train.Optimizer时，在这些集合中的Variable将被用为默认参数。

### 3.构建Graph
大多数Tensorflow程序都是从构建数据流图阶段开始，在这个阶段中，你唤起Tensorflow API函数构建新的操作节点以及Tensor对象(边)，并且将它们加入到tf.Graph实例中。Tensorflow提供一个默认图，隐式定义所有的API函数都在相同的上下文中，例如：
* 调用tf.constant(42.0)创建单独的tf.Operation产生数值42.0，并且将它加入到默认图中，返回一个tensor表示当前常数的数值。

大多数程序依赖各自的默认图，但一种情况是<a href ="https://www.tensorflow.org/programmers_guide/graphs?hl=zh-cn#programming_with_multiple_graphs"> Dealing with multiple graphs</a>。但是高层API，例如Estimator代替你来管理这些默认图，所以也会在训练以及评估的过程中自行创建不同的图。

### 4.操作命名
一个Graph对象为它包含的操作定义了命名空间。Tensorflow为每个操作自动选择一个独一无二的名字，但自行命名会使你的程序更易读，也更容易debug。

Tensorflow提供两种重命名operation的方式：
* 使用name参数：tf.constant(42.0, name="answer")


In [4]:
import tensorflow as tf
c_0 = tf.constant(0, name="c")  # => operation named "c"

# Already-used names will be "uniquified".
c_1 = tf.constant(2, name="c")  # => operation named "c_1"
c_1

<tf.Tensor 'c_5:0' shape=() dtype=int32>

* tf.name_scope方法允许你在特定的上下稳重给所有的操作创建前缀，如果某个scope的名字已经在当前上下文中使用，Tensorflow将自动在后面附上“_1”,"_2"这样

In [6]:
with tf.name_scope("outer"):
    c_2 = tf.constant(2, name="c")  # => operation named "outer/c"

  # Name scopes nest like paths in a hierarchical file system.
    with tf.name_scope("inner"):
        c_3 = tf.constant(3, name="c")  # => operation named "outer/inner/c"

  # Exiting a name scope context will return to the previous prefix.
    c_4 = tf.constant(4, name="c")  # => operation named "outer/c_1"

  # Already-used name scopes will be "uniquified".
    with tf.name_scope("inner"):
        c_5 = tf.constant(5, name="c")  # => operation named "outer/inner_1/c"

In [7]:
c_5

<tf.Tensor 'outer/inner_1/c:0' shape=() dtype=int32>

In [8]:
c_3

<tf.Tensor 'outer/inner/c:0' shape=() dtype=int32>

在TensorFlow可视化过程中，scope减少了可视化的复杂性

#### Note: 由tf.Operation隐式产生的tensor对象，自动用"OP_NAME":"i"的形式命名:
* Op_Name:产生该tensor的操作的名称
* i：在该操作产生一系列的输出中，该tensor的索引

### 5.在不同的硬件中放置操作
（略了，觉得买的起很多GPU的人不多

### 6.类Tensor对象
许多Tensorflow操作使用一个或者多个Tensor对象作为参数。例如，tf.matmul使用两个tensor对象，tf.add_n使用一列的tensor对象。为了方便起见，这些函数也接受类Tensor对象，并且隐式地转化为Tensor对象，类tensor对象包括以下几类：
* tf.Tensor
* tf.Variable
* numpy.ndarray
* list
* Python中的bool, float, int, str
同样，你也可以通过tf.register_tensor_conversion_function其他数据类型

### 7.在Session中执行Graph
Tensorflow提供了tf.Session类来表示客户程序之间的链接，一个tf.Session对象提供进入本地机器或者运行分布式Tensorflow的远程设备。它同样保存Graph的信息以便于你可以有效运行多次相同运算

#### 7.1创建tf.Session
如果你正在使用低层次的Tensorflow API，你可以按照以下方式创建Session：

In [12]:
# Create a default in-process session.
with tf.Session() as sess:
      sess.run(c_0)

或者创建一个远程session
* with tf.Session("grpc://example.org:2222"):
 

因为一个session拥有自己的物理资源(例如GPU和网络连接)，通常会使用上下文管理(在一个with模块中），当你退出某块时自动关闭session。在不使用with块的情况下创建一个session也是可能的，但是你需要显示的调用tf.Session.close来释放刚刚占用的所有资源。

#### Note：Estimator将自动管理session，但这些API接受target或者config参数，含义如下。

tf.Session.init接受三个可选择的参数：
* target：如果此处参数被留空的话，该session将只使用本地机器，然而，如果你指定grpc://URL到一个特定的Tensorflow服务器，那么该session具有该服务器上所有机器的进入权限。查看<a href="https://www.tensorflow.org/api_docs/python/tf/train/Server?hl=zh-cn">tf.train.Server</a>如何创建Tensorflow服务器。
* Graph：默认来说，一个新的session只可以运行当前Graph中包含的操作，如果你在你的程序中使用了多个图的话，你可以显示的定义一个tf.Graph来构建session
* config：该参数允许你指定一个 tf.ConfigProto来控制session的行为，包括
    * allow_soft_placement
    * cluster_def
    * graph_options.optimizer_options
    * gpu_options.allow_growth

#### 7.2 使用 tf.Session.run 来执行操作
tf.Session.run方法是执行tf.Operation或者估测tf.Tensor值的主要操作。你可以在tf.Session.run中传递一个或者多个tf.Operation或者tf.Tensor对象，Tensorflow会在需要计算结果的时候执行这些操作。

tf.Session.run要求你指定一系列需要取回的值，也就是决定返回值，可能是一个tf.Operation，tf.Tensor或者是类tensor对象。这些要取回的对象决定了整个tf.Graph图的子图，也就是该子图包含所有在取回值列的操作或者tensor对象。

In [13]:
x = tf.constant([[37.0, -23.0], [1.0, 4.0]])
w = tf.Variable(tf.random_uniform([2, 2]))
y = tf.matmul(x, w)
output = tf.nn.softmax(y)
init_op = w.initializer

with tf.Session() as sess:
  # Run the initializer on `w`.
    sess.run(init_op)

  # Evaluate `output`. `sess.run(output)` will return a NumPy array containing
  # the result of the computation.
    print(sess.run(output))

  # Evaluate `y` and `output`. Note that `y` will only be computed once, and its
  # result used both to return `y_val` and as an input to the `tf.nn.softmax()`
  # op. Both `y_val` and `output_val` will be NumPy arrays.
    y_val, output_val = sess.run([y, output])

[[9.9986076e-01 1.3921809e-04]
 [6.3378370e-01 3.6621630e-01]]


此外，tf.Session.run也可以用字典作为输入，映射tensor对象(尤其是针对placeholder类型)，到具体数值。

例如：
* sess.run(y, {x: [1.0, 2.0, 3.0]})

tf.Session.run也接受其他可选择的参数，使你可以指定调用，以及一个可选择的run_metadata参数允许你收集执行时的元参数，例如：

In [16]:
y = tf.matmul([[37.0, -23.0], [1.0, 4.0]], tf.random_uniform([2, 2]))

with tf.Session() as sess:
  # Define options for the `sess.run()` call.
    options = tf.RunOptions()
    options.output_partition_graphs = True
    options.trace_level = tf.RunOptions.FULL_TRACE

  # Define a container for the returned metadata.
    metadata = tf.RunMetadata()

    sess.run(y, options=options, run_metadata=metadata)

  # Print the subgraphs that executed on each device.
    print(metadata.partition_graphs)

  # Print the timings of each operation that executed.
    print(metadata.step_stats)

[node {
  name: "MatMul_1/a"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/device:CPU:0"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 2
          }
          dim {
            size: 2
          }
        }
        tensor_content: "\000\000\024B\000\000\270\301\000\000\200?\000\000\200@"
      }
    }
  }
}
node {
  name: "random_uniform_1/shape"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/device:CPU:0"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 2
          }
        }
        tensor_content: "\002\000\000\000\002\000\000\000"
      }
    }
  }
}
node {
  name: "random_uniform_1/RandomUniform"
  op: "RandomUniform"
  input: "random_uniform_1/shape"
  

### 8. 可视化图
这部分放在另一个<a href = "">visualization tutorial</a>啦